# Psychological Therapies - Monthly Reports

http://content.digital.nhs.uk/iaptmonthly
e.g. Monthly data file: http://content.digital.nhs.uk/catalogue/PUB21575

Historical data: http://content.digital.nhs.uk/iaptreports

See also the annual report: http://www.content.digital.nhs.uk/catalogue/PUB22110

## About
This notebook:
    
- downloads data from monthly reports, stores it in a sqlite database and retrieves it into a pandas dataframe
- loads in column metadata and represents it in a metadata dict
- explores several bespoke approaches to generating sentences from columns
- ends up with a generic cell list textualiser function that allows customised reporting

In [1]:
import pandas as pd
import re
import inflect
p = inflect.engine()

#http://stackoverflow.com/a/3847369
ls = lambda s: s[:1].lower() + s[1:] if s else ''

In [2]:
pd.set_option('display.max_colwidth',500)

In [ ]:
#metadata
url='http://content.digital.nhs.uk/media/20844/IAPT-month-metadata/xls/IAPT-month-metadata.xlsx'
!mkdir -p metadata/
!wget -P metadata/ http://content.digital.nhs.uk/media/20844/IAPT-month-metadata/xls/IAPT-month-metadata.xlsx

In [3]:
xl=pd.ExcelFile('metadata/IAPT-month-metadata.xlsx')
xl.sheet_names

['Front sheet',
 'Version history',
 'Monthly data measures',
 'Quarterly data measures',
 'IAPT v1.5 derivations',
 'Related DQ measures',
 'Glossary']

In [4]:
metadata=pd.read_excel('metadata/IAPT-month-metadata.xlsx',sheetname='Monthly data measures',skiprows=6)
metadata.head(5)

,Monthly measure reference number,Executive Summary (National),Monthly CSV data file field name,Related quarterly measure reference number,IC derivation reference numbers,Related DQ metric IDs,Description of measure (where possible measures are described in terms of the classes of information defined in NHS Data Dictionary),"Construction (For all measures ensure IC_Use_Pathway_Flag is ""Y"")",Notes
0,n/a,NaN,CCG,n/a,RE04,n/a,The organisation code of the CCG the measures relate to,R.IC_CCG,NaN
1,n/a,NaN,Provider,n/a,n/a,n/a,The organisation code (code of provider) the measures relate to,R.ORGCODEPROVIDER,NaN
2,n/a,NaN,GroupType,n/a,n/a,n/a,"The organisational group type the measures relate to (""England"", ""CCG"", ""Provider"" or ""CCG/Provider"")",Hardcoded during NHS Digital processing,NaN
3,n/a,NaN,Month,n/a,n/a,n/a,The reporting month,Month and year from reference table (linked on H.MONTH_ID),NaN
4,imm01,Referrals received,ReferralsReceived,iqm01,"HE01, RE01, RE02","1, 7, 8, 11, 14, 101, 102, 108, 109, 111, 112, 123",Referrals with a referral request received date in the month,Count of distinct(IC_PATHWAY_ID) where REFRECDATE is in the period,Monthly equivalent of line 1 of previous quarterly reports


In [5]:
shortcol={'monthlymeasureref':'Monthly measure reference number ',
          'execSummary':'Executive Summary (National)',
          'colname':'Monthly CSV data file field name',
          'quarterlymeasureref':'Related quarterly measure reference number',
          'refnum':'IC derivation reference numbers',
          'metricid':'Related DQ metric IDs',
          'desc':'Description of measure\n(where possible measures are described in terms of the classes of information defined in NHS Data Dictionary)',
          'construction':'Construction\n(For all measures ensure IC_Use_Pathway_Flag is "Y")',
          'notes':'Notes'}

In [6]:
metadataDesc=metadata[[shortcol['colname'],shortcol['desc']]].dropna()
metadataDesc.columns=['colname','desc']
metadataDict=dict(zip(metadataDesc['colname'], metadataDesc['desc']))

## Monthly Data File

In [7]:
import sqlite3
#!rm iapt.sqlite
con_iapt = sqlite3.connect("iapt.sqlite")

In [ ]:
#cursor = con_iapt.cursor()
#cursor.execute('DROP TABLE IF EXISTS monthly')
#con_iapt.commit()

In [ ]:
def db_loader(con,url):
    months=[]
    if len(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name='monthly'",con_iapt)):
        months=pd.read_sql_query('SELECT DISTINCT Month FROM monthly', con)['Month'].tolist()
    df=pd.read_csv(url, na_values='*')
    if df['Month'].unique()[0] not in months:
        df.set_index(['Month','CCG','Provider']).to_sql(con=con, name='monthly',if_exists='append')

In [ ]:
url='http://content.digital.nhs.uk/catalogue/PUB21431/IAPT-month-May-2016-final-data.csv'
db_loader(con_iapt,url)

In [ ]:
pd.read_sql_query("SELECT * FROM monthly WHERE CCG='10L'",con_iapt)

In [ ]:
#monthly data file example
url='http://content.digital.nhs.uk/catalogue/PUB21575/IAPT-month-jun-2016-final-data.csv'
db_loader(con_iapt,url)

In [ ]:
pd.read_sql_query("SELECT * FROM monthly WHERE CCG='10L'",con_iapt)

In [8]:
df_jun2016=pd.read_sql_query("SELECT * FROM monthly WHERE Month='Jun-16'",con_iapt)

In [9]:
ccgCode='10L'
df_jun2016[df_jun2016['CCG']==ccgCode].dropna(axis=1,how='all')

,Month,CCG,Provider,CCGName,ProviderName,GroupType,ReferralsReceived,SelfReferrals,FirstAssessment,FirstAssessment28days,...,MeanHISessions,MeanCBTSessions,MeanCouplesSessions,MeanOtherHISessions,MeanLiSessions,ImprovementRate,RecoveryRate,ReliableRecoveryRate,FirstTreatment6WeeksFinishedCourseRate,FirstTreatment18WeeksFinishedCourseRate
888,Jun-16,10L,R1F,NHS ISLE OF WIGHT CCG,ISLE OF WIGHT NHS TRUST,CCG/Provider,325.0,310.0,245.0,210.0,...,7.2,6.6,10.9,13.5,5.0,63.7,49.2,46.3,92.9,100.0
889,Jun-16,10L,RDYDL,NHS ISLE OF WIGHT CCG,PSYCHOLOGICAL THERAPIES SOUTHHAMPTON OFFICE,CCG/Provider,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,Jun-16,10L,all,NHS ISLE OF WIGHT CCG,all Providers,CCG,325.0,310.0,245.0,210.0,...,7.2,6.6,10.9,13.5,5.0,63.7,49.2,46.3,92.9,100.0


In [ ]:
# Column names
#df_jun2016.columns.tolist()
'''
 'FirstToSecondTreatmentOver28days',
 'FirstToSecondTreatmentOver90days',
 'MeanWaitEnteredTreatment',
 'MedianWaitEnteredTreatment',
 'MeanWaitFinishedCourse',
 'MedianWaitFinishedCourse',
 'FirstADSMOnly',
 'NoADSM',
 'PairedADSM',
 'FirstPHQOnly',
 'NoPHQ',
 'PairedPHQ',
 'PairedADSMPHQ',
 'OffSickPay',
 'AptFinishedCourseTreatment',
 'MeanAptFinishedCourseTreatment',
 'MedianAptFinishedCourseTreatment',
 'MaxAptFinishedCourseTreatment',
 'MinAptFinishedCourseTreatment',
 'OpenReferralNoActivity60days',
 'OpenReferralNoActivity61to90days',
 'OpenReferralNoActivity91to120days',
 'OpenReferralNoActivityOver120days',
 'NotCaseness',
 'Recovery',
 'Improvement',
 'Deterioration',
 'NoReliableChange',
 'ReliableRecovery',
 'SecondTreatment',
 'MeanHISessions',
 'MeanCBTSessions',
 'MeanBPDSessions',
 'MeanCounsellingSessions',
 'MeanIPTSessions',
 'MeanCouplesSessions',
 'MeanBehavActSessions',
 'MeanOtherHISessions',
 'MeanLiSessions',
 'ImprovementRate',
 'RecoveryRate',
 'ReliableRecoveryRate',
 'FirstTreatment6WeeksFinishedCourseRate',
 'FirstTreatment18WeeksFinishedCourseRate'
 '''
pass

## Reporting Utility Functions

In [10]:
def report_rows(df,ccg,provider='all',period=None):
    if provider!='all':
        rows=df[(df['CCG']==ccg) & (df['Provider']==provider)]
    else:
        rows=df[(df['CCG']==ccg)]
    #period is a 2 tuple: (colname,[periods])
    if period is not None:
        if isinstance(period[1],str): period= (period[0], [period[1]] )
        rows=rows[rows[period[0]].isin(period[1])]
    return rows

def report_referral(df,series_reporter,ccg,provider='all',period=None):
    rows=report_rows(df,ccg,provider,period)
    tmp=rows[['CCG','Provider']][:]
    tmp['txt']= rows.apply(series_reporter,axis=1)
    return tmp

def report_referrals(df,series_reporters,ccg,provider='all',period=None):
    tmp=pd.DataFrame()
    for series_reporter in series_reporters:
        tmp=pd.concat([tmp,report_referral(df,series_reporter,ccg,provider=provider,period=period)])
    return tmp

def printer(df):
    df.apply(lambda x:print(x['txt']),axis=1)

## Bespoke Reporting

Getting a feel for structures for reporting on differnt columns and groups of related columns.

In [11]:
df=df_jun2016
def series_referrals(row):
    txt='''
In {period}, across the {ccg}, {provider} received {ReferralsReceived} referrals,
of which {SelfReferrals} were self-referrals.
    '''.format(period=row['Month'],
               ccg=row['CCGName'],
               provider=row['ProviderName'],
               ReferralsReceived=row['ReferralsReceived'],
               SelfReferrals=row['SelfReferrals']
              )
    return txt

In [12]:
printer(report_referral(df,series_referrals,'10L','R1F'))


In Jun-16, across the NHS ISLE OF WIGHT CCG, ISLE OF WIGHT NHS TRUST received 325.0 referrals,
of which 310.0 were self-referrals.
    


In [13]:
def series_referrals2(row):
    txt='''
In {period}, across the {ccg}, {provider} received {ReferralsReceived} referrals ({md1}), \
of which {SelfReferrals} were self-referrals ({md2}).
    '''.format(period=row['Month'],
               ccg=row['CCGName'],
               provider=row['ProviderName'],
               ReferralsReceived=row['ReferralsReceived'],
               SelfReferrals=row['SelfReferrals'],
               md1=ls(metadataDict['ReferralsReceived']),
               md2=ls(metadataDict['SelfReferrals'])
              )
    return txt

In [14]:
printer(report_referral(df,series_referrals2,'10L','R1F'))


In Jun-16, across the NHS ISLE OF WIGHT CCG, ISLE OF WIGHT NHS TRUST received 325.0 referrals (referrals with a referral request received date in the month), of which 310.0 were self-referrals (referrals with a referral request received date in the month with a source of referral for mental health of 'self').
    


In [15]:
#Generate some custom labels corresponding to column names
lookups={
    'FirstAssessment':('first assessments',''),
    'FirstAssessment28days':('first assessments','within 28 days'),
    'FirstAssessment29to56days':('first assessments','between 29 and 56 days'),
    'FirstAssessment57to90days':('first assessments','between 57 and 90 days'),
    'FirstAssessmentOver90days':('first assessments','over 90 days'),
    'EndedBeforeAssessment':('ended before assessment',''),
    'WaitingForAssessment':('waiting for assessment',''),
    'WaitingForAssessmentOver90days':('waiting for assessment','over 90 days'),
    'FirstTreatment':('first treatments',''),
    'FirstTreatment28days':('first treatments','within 28 days'),
    'FirstTreatment29to56days':('first treatments','between 29 and 56 days'),
    'FirstTreatment57to90days':('first treatments','between 57 and 90 days'),
    'FirstTreatmentOver90days':('first treatments','between 57 and 90 days'),
    'FirstTreatment6Weeks':('first treatments','within 6 weeks'),
    'FirstTreatment18Weeks':('first treatments','within 18 weeks'),
    'FirstTreatment6WeeksFinishedCourse':('first treatments','finished course within 6 weeks'),
    'FirstTreatment18WeeksFinishedCourse':('first treatments','finished course within 18 weeks'),
    'EndedBeforeTreatment':('ended before treatment',''),
    'WaitingForTreatment':('waiting for treatment',''),
    'WaitingForTreatment0to2weeks':('waiting for treatment','for 0 to 2 weeks'),
    'WaitingForTreatment0to4weeks':('waiting for treatment','for 0 to 4 weeks'),
    'WaitingForTreatment0to6weeks':('waiting for treatment','for 0 to 6 weeks'),
    'WaitingForTreatment0to12weeks':('waiting for treatment','for 0 to 12 weeks'),
    'WaitingForTreatment0to18weeks':('waiting for treatment','for 0 to 18 weeks'),
    'WaitingForTreatmentOver18weeks':('waiting for treatment','for over 18 weeks'),
    'WaitingForTreatmentOver90days':('waiting for treatment','for over 90 days'),
    'EndedReferrals':('referrals ended',''),
 'EndedNotSuitable':('referrals ended','not suitable'),
 'EndedSignposted':('referrals ended','signposted'),
 'EndedMutualAgreement':('referrals ended','by mutual agreement'),
 'EndedReferredElsewhere':('referrals ended','referred elsewhere'),
 'EndedDeclined':('referrals ended','declined'),
 'EndedDeceasedAssessedOnly':('referrals ended','deceased (assessed only)'),
 'EndedUnknownAssessedOnly':('referrals ended','unknown (assessed only)'),
 'EndedSteppedUp':('referrals ended','stepped up'),
 'EndedSteppedDown':('referrals ended','steppd down'),
 'EndedCompleted':('referrals ended','completed'),
 'EndedDroppedOut':('referrals ended','dropped out'),
 'EndedReferredNonIAPT':('referrals ended','referred non-IAPT'),
 'EndedDeceasedTreated':('referrals ended','deceased (treated)'),
 'EndedUnknownTreated':('referrals ended','unknown (treated)'),
 'EndedInvalid':('referrals ended','invalid'),
 'EndedNoReasonRecorded':('referrals ended','no reason recorded'),
 'EndedSeenNotTreated':('referrals ended','seen but not treated'),
 'EndedTreatedOnce':('referrals ended','treated once'),
 'FinishedCourseTreatment':('referrals ended','finished the course of treatment'),
 'EndedNotSeen':('referrals ended','not seen'),
     'Appointments':('appointments',''),
 'AptCancelledPatient':('appointments','cancelled by patient'),
 'AptDNA':('appointments','did not attend (DNA)'),
 'AptCancelledProvider':('appointments','cancelled by provider'),
 'AptAttended':('appointments','attended'),
 'AptAttendedLate':('appointments','attended late'),
 'AptLateNotSeen':('appointments','late and not seen'),
 'GuideSelfHelpBookApts':('aptType',''),
 'NonGuideSelfHelpBookApts':('aptType',''),
 'GuideSelfHelpCompApts':('aptType',''),
 'NonGuideSelfHelpCompApts':('aptType',''),
 'BehavActLIApts':('aptType',''),
 'StructPhysActApts':('aptType',''),
 'AntePostNatalCounselApts':('aptType',''),
 'PsychoEducPeerSuppApts':('aptType',''),
 'OtherLIApts':('aptType',''),
 'EmploySuppLIApts':('aptType',''),
 'AppRelaxApts':('aptType',''),
 'BehavActHIApts':('aptType',''),
 'CoupleTherapyDepApts':('aptType',''),
 'CollabCareApts':('aptType',''),
 'CounselDepApts':('aptType',''),
 'BPDApts':('aptType',''),
 'EyeMoveDesenReproApts':('aptType',''),
 'MindfulApts':('aptType',''),
 'OtherHIApts':('aptType',''),
 'EmploySuppHIApts':('aptType',''),
 'CBTApts':('aptType',''),
 'IPTApts':('aptType','')
}

In [16]:
#DONE
#'FirstAssessment','FirstAssessment28days', 'FirstAssessment29to56days', 'FirstAssessment57to90days', 'FirstAssessmentOver90days', 'EndedBeforeAssessment','WaitingForAssessment', 'WaitingForAssessmentOver90days'
def series_assessment_all(row):
    txt='''
In all, there were {FirstAssessment} first assessments:
    
    - {FirstAssessment28days} within 28 days;
    - {FirstAssessment29to56days} between 29 and 56 days;
    - {FirstAssessment57to90days} between 57 and 90 days;
    - {FirstAssessmentOver90days} over 90 days;

{EndedBeforeAssessment} ended before assessment, {WaitingForAssessment} are waiting for assessment,
 and {WaitingForAssessmentOver90days} have been waiting for assessment for over 90 days.
    '''.format(
        FirstAssessment=row['FirstAssessment'],
        FirstAssessment28days=row['FirstAssessment28days'],
        FirstAssessment29to56days=row['FirstAssessment29to56days'],
        FirstAssessment57to90days=row['FirstAssessment57to90days'],
        FirstAssessmentOver90days=row['FirstAssessmentOver90days'],
        EndedBeforeAssessment=row['EndedBeforeAssessment'],
        WaitingForAssessment=row['WaitingForAssessment'],
        WaitingForAssessmentOver90days=row['WaitingForAssessmentOver90days']
    )
    return txt

def series_assessment_nonzero(row):
    if row['FirstAssessment']==0:
        txt='There were no {} ({}).'.format(lookups['FirstAssessment'][0], ls(metadataDict['FirstAssessment']))
    else:
        txt='''In all, there were {issue} {typ}: '''.format(issue=row['FirstAssessment'],typ=lookups['FirstAssessment'][0])
        for i in ['FirstAssessment28days','FirstAssessment29to56days','FirstAssessment57to90days','FirstAssessmentOver90days']:
            if row[i]>0: txt='{txt}\n\t- {cnt} {period}'.format(txt=txt,cnt=row[i],period=lookups[i][1])
    tl=[]
    txt=txt+'\n'
    for i in ['EndedBeforeAssessment','WaitingForAssessment','WaitingForAssessmentOver90days']:
        if row[i]>0: tl.append('{cnt} {typ} {period}'.format(cnt=row[i],typ=lookups[i][0],period=lookups[i][1]).strip())
    if len(tl):
        txt='{txt}\n{tl}'.format(txt=txt,tl=', '.join(tl))
    return txt+'\n'

In [17]:
printer(report_referral(df,series_assessment_nonzero,'10L','R1F'))

In all, there were 245.0 first assessments: 
	- 210.0 within 28 days
	- 35.0 between 29 and 56 days

120.0 ended before assessment, 190.0 waiting for assessment, 10.0 waiting for assessment over 90 days



In [18]:
#DONE
def series_treatment_nonzero(row,metadata=True):
    typ=['FirstTreatment28days','FirstTreatment29to56days','FirstTreatment57to90days',
                  'FirstTreatmentOver90days','FirstTreatment6Weeks','FirstTreatment18Weeks']
    if row['FirstTreatment']==0: txt= 'There were no {} ({}).'.format(lookups['FirstTreatment'][0],ls(metadataDict['FirstTreatment']))
    else:
        txt='''In all, there were {issue} {typ} ({md}): '''.format(issue=row['FirstTreatment'],typ=lookups['FirstTreatment'][0],md=ls(metadataDict['FirstTreatment']))
        for i in typ:
            if row[i]>0: txt='{txt}\n\t- {cnt} {period}'.format(txt=txt,cnt=row[i],period=lookups[i][1])

    txt=txt+'\n\n'
    for i in ['FirstTreatment6WeeksFinishedCourse','FirstTreatment18WeeksFinishedCourse']:
        if row[i]>0:
            txt=txt+'{cnt} {period}{md}.\n'.format(cnt=row[i],
                                                  period=lookups[i][1],
                                                  md=' ({})'.format(ls(metadataDict[i])) if metadata else ''
                                                 )
    for i in ['EndedBeforeTreatment']:
        if row[i]>0: txt=txt+'{cnt} {period}{md}.\n'.format(cnt=row[i],
                                                           period=lookups[i][0],
                                                           md=' ({})'.format(ls(metadataDict[i])) if metadata else ''
                                                          )
    return txt+'\n'

In [19]:
printer(report_referral(df,series_treatment_nonzero,'10L','R1F'))

In all, there were 255.0 first treatments (referrals with a first treatment appointment (entered treatment) in the month): 
	- 215.0 within 28 days
	- 40.0 between 29 and 56 days
	- 250.0 within 6 weeks
	- 255.0 within 18 weeks

170.0 finished course within 6 weeks (referrals that finished a course of treatment in the month waiting 42 days or less for first treatment).
180.0 finished course within 18 weeks (referrals that finished a course of treatment in the month waiting 126 days or less for first treatment).
105.0 ended before treatment (referrals with an end date in the month before first treatment).




In [20]:
#DONE
def series_waiting_nonzero(row):
    typ=['WaitingForTreatment0to2weeks','WaitingForTreatment0to4weeks','WaitingForTreatment0to6weeks',
                  'WaitingForTreatment0to12weeks','WaitingForTreatment0to18weeks','WaitingForTreatmentOver18weeks',
                  'WaitingForTreatmentOver90days']
    if row['WaitingForTreatment']==0: txt= 'There were no {} ({}).'.format(lookups['WaitingForTreatment'][0],ls(metadataDict['WaitingForTreatment']))
    else:
        txt='''In all, there were {issue} {typ} ({md}): '''.format(issue=row['WaitingForTreatment'],typ=lookups['WaitingForTreatment'][0],md=ls(metadataDict['WaitingForTreatment']))
        for i in typ:
            if row[i]>0: txt='{txt}\n\t- {cnt} {period}'.format(txt=txt,cnt=row[i],period=lookups[i][1])
    return txt+'\n'

In [21]:
printer(report_referral(df,series_waiting_nonzero,'10L','R1F'))

In all, there were 170.0 waiting for treatment (referrals yet to have a first treatment at the end of the month): 
	- 120.0 for 0 to 2 weeks
	- 155.0 for 0 to 4 weeks
	- 165.0 for 0 to 6 weeks
	- 165.0 for 0 to 12 weeks
	- 165.0 for 0 to 18 weeks



In [29]:
#DONE
def series_appointments_nonzero(row):
    typ=['AptAttended','AptCancelledPatient','AptCancelledProvider',
                  'AptDNA','AptAttendedLate','AptLateNotSeen']
    df=row[typ].T.dropna()
    if row['Appointments']==0: txt= 'There were no {}.'.format(lookups['Appointments'][0],ls(metadataDict['Appointments']))
    else:
        txt='''In all, there were {issue} {typ} ({md}), of which:'''.format(issue=row['Appointments'],typ=lookups['Appointments'][0],md=ls(metadataDict['Appointments']))
        for i,trow in df.sort_values(ascending=False).iteritems():
            if trow>0: txt='{txt}\n\t- {cnt} {period}'.format(txt=txt,cnt=trow,period=lookups[i][1])
    return txt+'\n'

In [30]:
printer(report_referral(df,series_appointments_nonzero,'10L','R1F'))

In all, there were 1195.0 appointments (appointments in the month), of which:
	- 1090.0 attended
	- 45.0 did not attend (DNA)
	- 30.0 attended late
	- 30.0 cancelled by patient



In [31]:
#DONE
def series_appointmentTypes_nonzero(row):
    typ=[ 'GuideSelfHelpBookApts','NonGuideSelfHelpBookApts','GuideSelfHelpCompApts','NonGuideSelfHelpCompApts',
        'BehavActLIApts','StructPhysActApts','AntePostNatalCounselApts','PsychoEducPeerSuppApts',
        'OtherLIApts','EmploySuppLIApts','AppRelaxApts','BehavActHIApts','CoupleTherapyDepApts',
        'CollabCareApts','CounselDepApts','BPDApts','EyeMoveDesenReproApts','MindfulApts', 'OtherHIApts',
        'EmploySuppHIApts','CBTApts','IPTApts']
    txt='The appointments types included:'
    for i in typ:
        if row[i]>0: txt='{txt}\n\t- {typ} appointments: {cnt}'.format(txt=txt,cnt=row[i],typ=metadataDict[i].split('Appointments in the month')[0])
    return txt+'\n'

In [32]:
printer(report_referral(df,series_appointmentTypes_nonzero,'10L','R1F'))

The appointments types included:
	- Number of Guided Self Help (Book) treatment  appointments: 10.0
	- Number of Non - Guided Self Help (Book) treatment  appointments: 10.0
	- Number of Guided Self Help (Computer) treatment  appointments: 60.0
	- Number of Behavioural Activation (Low Intensity) treatment  appointments: 45.0
	- Number of Psychoeducational Peer Support treatment  appointments: 90.0
	- Number of Other Low Intensity treatment  appointments: 135.0
	- Number of Couples Therapy for Depression treatment  appointments: 20.0
	- Number of Eye Movement Desensitisation Reprocessing treatment  appointments: 55.0
	- Number of Cognitive Behaviour Therapy (CBT) treatment  appointments: 695.0
	- Number of Interpersonal Psycho Therapy (IPT) treatment  appointments: 30.0



In [33]:
#DONE
def series_appointmentTypes2_nonzero(row):
    typ=[ 'GuideSelfHelpBookApts','NonGuideSelfHelpBookApts','GuideSelfHelpCompApts','NonGuideSelfHelpCompApts',
        'BehavActLIApts','StructPhysActApts','AntePostNatalCounselApts','PsychoEducPeerSuppApts',
        'OtherLIApts','EmploySuppLIApts','AppRelaxApts','BehavActHIApts','CoupleTherapyDepApts',
        'CollabCareApts','CounselDepApts','BPDApts','EyeMoveDesenReproApts','MindfulApts', 'OtherHIApts',
        'EmploySuppHIApts','CBTApts','IPTApts']
    df=row[typ].T.dropna()
    txt='The appointment types included:'
    for i,trow in df.sort_values(ascending=False).iteritems():
        if trow>0: txt='{txt}\n\t- {typ} appointments: {cnt}'.format(txt=txt,cnt=trow,
                                                                     typ=metadataDict[i].split('Appointments in the month')[0].replace('Number of','').strip())
    return txt+'\n'

printer(report_referral(df,series_appointmentTypes2_nonzero,'10L','R1F'))

The appointment types included:
	- Cognitive Behaviour Therapy (CBT) treatment appointments: 695.0
	- Other Low Intensity treatment appointments: 135.0
	- Psychoeducational Peer Support treatment appointments: 90.0
	- Guided Self Help (Computer) treatment appointments: 60.0
	- Eye Movement Desensitisation Reprocessing treatment appointments: 55.0
	- Behavioural Activation (Low Intensity) treatment appointments: 45.0
	- Interpersonal Psycho Therapy (IPT) treatment appointments: 30.0
	- Couples Therapy for Depression treatment appointments: 20.0
	- Non - Guided Self Help (Book) treatment appointments: 10.0
	- Guided Self Help (Book) treatment appointments: 10.0



In [34]:
printer(report_referrals(df,[series_referrals,series_assessment_nonzero,series_treatment_nonzero,series_waiting_nonzero,
                         series_ended_nonzero,series_appointments_nonzero,series_appointmentTypes2_nonzero],'10L','R1F'))



In Jun-16, across the NHS ISLE OF WIGHT CCG, ISLE OF WIGHT NHS TRUST received 325.0 referrals,
of which 310.0 were self-referrals.
    
In all, there were 245.0 first assessments: 
	- 210.0 within 28 days
	- 35.0 between 29 and 56 days

120.0 ended before assessment, 190.0 waiting for assessment, 10.0 waiting for assessment over 90 days

In all, there were 255.0 first treatments (referrals with a first treatment appointment (entered treatment) in the month): 
	- 215.0 within 28 days
	- 40.0 between 29 and 56 days
	- 250.0 within 6 weeks
	- 255.0 within 18 weeks

170.0 finished course within 6 weeks (referrals that finished a course of treatment in the month waiting 42 days or less for first treatment).
180.0 finished course within 18 weeks (referrals that finished a course of treatment in the month waiting 126 days or less for first treatment).
105.0 ended before treatment (referrals with an end date in the month before first treatment).


In all, there were 170.0 waiting for treatmen

It would be much more convenient to specify a generic reporting function and then pass values into it...

## Generic Reporting

Generic functions for reporting on the contents of one or more cells.

In [397]:
def series_lister_item(numFirst,cnt,typ,prefix,suffix,extractor,replacer,lower):
    if extractor is not None:
        rs=re.search(extractor,typ)
        if rs:
            typ=rs.group(1)
    if replacer is not None:
        typ=re.sub(replacer[0],replacer[1],typ)
    if numFirst:
        txt='{prefix}{cnt} {typ}{suffix}'.format(cnt=cnt,typ=ls(typ) if lower else typ,
                                                      prefix=prefix,suffix=suffix)
    else:
        txt='{prefix}{typ}: {cnt}{suffix}'.format(cnt=cnt,typ=typ,
                                                       prefix=prefix,suffix=suffix)
    return txt
    
def series_lister_nonzero(row,cols,metadata,sort=False,numFirst=False,prefix='\n',suffix='',
                          header='',extractor=None,replacer=None,lower=False):
    txt=header
    if sort:
        df=row[cols].T.dropna()
        for i,trow in df.sort_values(ascending=False).iteritems():
            typ=metadata[i]
            if pd.notnull(trow):
                txt='{txt}{item}'.format(txt=txt,item=series_lister_item(numFirst,trow,typ,prefix,suffix,extractor,replacer,lower))
    else:
        for i in cols:
            typ=metadata[i]
            if pd.notnull(row[i]):
                txt='{txt}{item}'.format(txt=txt,item=series_lister_item(numFirst,row[i],typ,prefix,suffix,extractor,replacer,lower))
                
    return txt+'\n'

def generic_reporter(df,setup,ccg,provider='all',reference=None,period=None):
    rows=report_rows(df,ccg,provider,period)
    cols=['CCG','Provider']
    if isinstance(reference, str): cols.append(reference)

    tmp=rows[cols][:]
    tmp['txt']= rows.apply(lambda x: series_lister_nonzero(x,**setup),axis=1)
    
    #Need to identify multiple reports
    if reference is None and len(rows)>1: reference=True
    if reference: tmp['txt']=tmp.apply(lambda x: '[{}: {}{}] {}'.format(x['CCG'],x['Provider'],
                                                                        '' if not isinstance(reference, str) else ', {}'.format(x[reference]),
                                                                        x['txt']),axis=1)

    return tmp

In [398]:
#Groups of related columns to be reported on separately
openreferrals=['OpenReferralNoActivity60days',
 'OpenReferralNoActivity61to90days',
 'OpenReferralNoActivity91to120days',
 'OpenReferralNoActivityOver120days'
]

waiting=['WaitingForTreatment0to2weeks','WaitingForTreatment0to4weeks','WaitingForTreatment0to6weeks',
                  'WaitingForTreatment0to12weeks','WaitingForTreatment0to18weeks','WaitingForTreatmentOver18weeks',
                  'WaitingForTreatmentOver90days']

appts=['AptAttended','AptCancelledPatient','AptCancelledProvider',
                  'AptDNA','AptAttendedLate','AptLateNotSeen']

ended=['EndedNotSuitable','EndedSignposted','EndedMutualAgreement',
                  'EndedReferredElsewhere','EndedDeclined','EndedDeceasedAssessedOnly',
                  'EndedUnknownAssessedOnly','EndedSteppedUp','EndedSteppedDown','EndedCompleted',
                  'EndedDroppedOut','EndedReferredNonIAPT','EndedDeceasedTreated',
                  'EndedUnknownTreated','EndedInvalid','EndedNoReasonRecorded',
                  'EndedSeenNotTreated','EndedTreatedOnce', 'FinishedCourseTreatment','EndedNotSeen'
                 ]

apptTypes=[ 'GuideSelfHelpBookApts','NonGuideSelfHelpBookApts','GuideSelfHelpCompApts','NonGuideSelfHelpCompApts',
        'BehavActLIApts','StructPhysActApts','AntePostNatalCounselApts','PsychoEducPeerSuppApts',
        'OtherLIApts','EmploySuppLIApts','AppRelaxApts','BehavActHIApts','CoupleTherapyDepApts',
        'CollabCareApts','CounselDepApts','BPDApts','EyeMoveDesenReproApts','MindfulApts', 'OtherHIApts',
        'EmploySuppHIApts','CBTApts','IPTApts']


firstTreatment=['FirstTreatment28days','FirstTreatment29to56days','FirstTreatment57to90days',
                  'FirstTreatmentOver90days','FirstTreatment6Weeks','FirstTreatment18Weeks']

firstAssessment=[    'FirstAssessment','FirstAssessment28days','FirstAssessment29to56days',
    'FirstAssessment57to90days', 'FirstAssessmentOver90days', 'EndedBeforeAssessment', 'WaitingForAssessment',
    'WaitingForAssessmentOver90days',
]


outcomes=['NotCaseness','Recovery','Improvement', 'Deterioration', 'NoReliableChange', 'ReliableRecovery']

endcourse=['FirstADSMOnly', 'NoADSM', 'PairedADSM','FirstPHQOnly',
 'NoPHQ','PairedPHQ', 'PairedADSMPHQ', 'OffSickPay']

longwait=['FirstToSecondTreatmentOver28days',
 'FirstToSecondTreatmentOver90days']

avwaiting=[ 'MeanWaitEnteredTreatment','MedianWaitEnteredTreatment', 'MeanWaitFinishedCourse',
           'MedianWaitFinishedCourse']


quickfinish=[ 'FirstTreatment6WeeksFinishedCourseRate','FirstTreatment18WeeksFinishedCourseRate']

outcomerate=[ 'ImprovementRate', 'RecoveryRate', 'ReliableRecoveryRate']

morethan2=['MeanAptFinishedCourseTreatment', 'MedianAptFinishedCourseTreatment',
 'MaxAptFinishedCourseTreatment','MinAptFinishedCourseTreatment',
           'MeanCBTSessions', 'MeanBPDSessions', 'MeanCounsellingSessions', 'MeanIPTSessions',
 'MeanCouplesSessions', 'MeanBehavActSessions', 'MeanOtherHISessions', 'MeanLiSessions',
          'AptFinishedCourseTreatment','MeanHISessions']

In [399]:
#dirtyMetadataDataCleaner
kk=[k for k in metadataDict.keys()]
for k in ['WaitingForTreatment0to2weeks','WaitingForTreatment0to4weeks','WaitingForTreatment0to6weeks',
                  'WaitingForTreatment0to12weeks','WaitingForTreatment0to18weeks','WaitingForTreatmentOver18weeks',
                  'WaitingForTreatmentOver90days','MeanLiSessions']:
    for i in kk:
        if i.lower()==k.lower():metadataDict[k]=metadataDict[i]
    

In [400]:
#Report focus
ccg='10L'
provider='R1F'

In [450]:
#Literal printing
print('Report for CCG {} and provider {}'.format(ccg,provider))
      
printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['Appointments'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'sort':True,'cols':apptTypes,'numFirst':True,
                          'header':'Appointment types:',
                          'prefix':"\n\t- ",'suffix':' appointments;',
                         'extractor':"Number of (.*) Appointments in the month where attended or did not attend code is  'attended on time' or 'arrived late but was seen'"},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))



printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':firstAssessment,'sort':False,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))



printer(generic_reporter(df,{'metadata':metadataDict,'header':'Proportion of referrals that finished a course of treatment  (end date in the month and a minimum of two attended treatment appointments in the course of the referral) that waited:',
                          'extractor':"Proportion of referrals that finished a course of treatment  \(end date in the month and a minimum of two attended treatment appointments in the course of the referral\) that waited (.*)$",
                          'cols':quickfinish,'sort':False,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))

printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['FirstTreatment'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))

printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['WaitingForTreatment'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))

printer(generic_reporter(df,{'metadata':metadataDict,'header':'Referrals that entered treatment in the month that waited:',
                          'extractor':'Referrals that entered treatment in the month that waited (.*)',
                          'cols':firstTreatment,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))



printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['EndedReferrals'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))

printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['EndedBeforeTreatment'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'The ended referrals included:',
                          'extractor':"^Referrals with an end date in the month - Improving Access to Psychological Therapies care spell end code is '?(.*)'+?$",
                          'cols':ended,'sort':True,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'Referrals with an end date in the month that finished a course of treatment:',
                          'extractor':"Referrals with an end date in the month that finished a course of treatment (.*)$",
                          'cols':outcomes,'sort':True,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['FirstTreatment6WeeksFinishedCourse','FirstTreatment18WeeksFinishedCourse'],
                          'sort':False,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))

printer(generic_reporter(df,{'metadata':metadataDict,'header':'Referrals with an end date in the month that finished a course of treatment:',
                          'extractor':"Referrals with an end date in the month that finished a course of treatment (.*)$",
                          'cols':endcourse,'sort':True,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))



printer(generic_reporter(df,{'metadata':metadataDict,'header':'Proportion of referrals that finished a course of treatment (end date in the month and a minimum of two attended treatment appointments in the course of the referral):',
                          'extractor':"Proportion of referrals that finished a course of treatment \(end date in the month and a minimum of two attended treatment appointments in the course of the referral\) (.*)$",
                          'cols':outcomerate,'sort':False,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'Waiting periods for referrals yet to have first treatment:',
                          'extractor':"Referrals yet to have a first treatment who have been waiting (.*) at the end of the month",
                          'cols':waiting,'sort':False,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))

printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['SecondTreatment'],'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':longwait,'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


printer(generic_reporter(df,{'metadata':metadataDict,'header':'For referrals with an end date in the month after at least two treatments:',
                          'extractor':"(.*) for referrals with an end date in the month after at least two treatments",
                          'cols':morethan2,'sort':True,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))



 
printer(generic_reporter(df,{'metadata':metadataDict,'header':'Waiting time in days between the referral received date and the date of first therapeutic session:',
                          'replacer':(" waiting time in days between the (referral request|referral) received date and the date of first therapeutic session",''),
                          'cols':avwaiting,'sort':False,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))



printer(generic_reporter(df,{'metadata':metadataDict,'cols':openreferrals,'numFirst':True,
                          'header':'Open referrals with no activity at end of the month:',
                          'extractor':"Open referrals with no activity at end of the month (.*)$",
                          'prefix':"\n\t- ",'suffix':'.'},ccg,provider))

Report for CCG 10L and provider R1F

Appointments in the month: 1195.0.

Appointment types:
	- 695.0 Cognitive Behaviour Therapy (CBT) treatment appointments;
	- 135.0 Other Low Intensity treatment appointments;
	- 90.0 Psychoeducational Peer Support treatment appointments;
	- 60.0 Guided Self Help (Computer) treatment appointments;
	- 55.0 Eye Movement Desensitisation Reprocessing treatment appointments;
	- 45.0 Behavioural Activation (Low Intensity) treatment appointments;
	- 30.0 Interpersonal Psycho Therapy (IPT) treatment appointments;
	- 20.0 Couples Therapy for Depression treatment appointments;
	- 10.0 Non - Guided Self Help (Book) treatment appointments;
	- 10.0 Guided Self Help (Book) treatment appointments;

Appointment attendance:
	- 1090.0 attended on time.
	- 30.0 cancelled appointments - patient.
	- 45.0 did not attend.
	- 30.0 arrived late but was seen.


Referrals with first assessment in the month: 245.0.
Referrals that waited fewer than 29 days for first assessment: 

In [40]:
#List append
ddl=[]
ddl.append(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['Appointments'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


ddl.append(generic_reporter(df,{'metadata':metadataDict,'sort':True,'cols':apptTypes,'numFirst':True,
                          'header':'Appointment types:',
                          'prefix':"\n\t- ",'suffix':' appointments;',
                         'extractor':"Number of (.*) Appointments in the month where attended or did not attend code is  'attended on time' or 'arrived late but was seen'"},ccg,provider))


ddl.append(generic_reporter(df,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))
for i in ddl:
    printer(i)


Appointments in the month: 1195.0.

Appointment types:
	- 695.0 Cognitive Behaviour Therapy (CBT) treatment appointments;
	- 135.0 Other Low Intensity treatment appointments;
	- 90.0 Psychoeducational Peer Support treatment appointments;
	- 60.0 Guided Self Help (Computer) treatment appointments;
	- 55.0 Eye Movement Desensitisation Reprocessing treatment appointments;
	- 45.0 Behavioural Activation (Low Intensity) treatment appointments;
	- 30.0 Interpersonal Psycho Therapy (IPT) treatment appointments;
	- 20.0 Couples Therapy for Depression treatment appointments;
	- 10.0 Non - Guided Self Help (Book) treatment appointments;
	- 10.0 Guided Self Help (Book) treatment appointments;

Appointment attendance:
	- 1090.0 attended on time.
	- 30.0 cancelled appointments - patient.
	- 45.0 did not attend.
	- 30.0 arrived late but was seen.



In [41]:
#Dataframe report items
ddf=pd.DataFrame()
def ddf_build(row):
    global ddf
    ddf=pd.concat([ddf,row])
    return

def rowprinter(row):
    print(row['txt'])
    return
    
ddf_build(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['Appointments'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


ddf_build(generic_reporter(df,{'metadata':metadataDict,'sort':True,'cols':apptTypes,'numFirst':True,
                          'header':'Appointment types:',
                          'prefix':"\n\t- ",'suffix':' appointments;',
                         'extractor':"Number of (.*) Appointments in the month where attended or did not attend code is  'attended on time' or 'arrived late but was seen'"},ccg,provider))


ddf_build(generic_reporter(df,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))
#for i in ddl:
#    printer(i)
ddf.apply(rowprinter,axis=1)
pass


Appointments in the month: 1195.0.

Appointment types:
	- 695.0 Cognitive Behaviour Therapy (CBT) treatment appointments;
	- 135.0 Other Low Intensity treatment appointments;
	- 90.0 Psychoeducational Peer Support treatment appointments;
	- 60.0 Guided Self Help (Computer) treatment appointments;
	- 55.0 Eye Movement Desensitisation Reprocessing treatment appointments;
	- 45.0 Behavioural Activation (Low Intensity) treatment appointments;
	- 30.0 Interpersonal Psycho Therapy (IPT) treatment appointments;
	- 20.0 Couples Therapy for Depression treatment appointments;
	- 10.0 Non - Guided Self Help (Book) treatment appointments;
	- 10.0 Guided Self Help (Book) treatment appointments;

Appointment attendance:
	- 1090.0 attended on time.
	- 30.0 cancelled appointments - patient.
	- 45.0 did not attend.
	- 30.0 arrived late but was seen.



In [42]:
#Tidy things up by creating objects for each report?
class storyBlock():
    def __init__(self):
        self.ddf = pd.DataFrame()
    
    def add(self,row):
        if type(row)==dict:
            row=pd.DataFrame([row])
        self.ddf=pd.concat([self.ddf,row])
        
    def rowprinter(self,row):
         print(row['txt'])
    
    def printer(self):
        self.ddf.apply(rowprinter,axis=1)
        
    def printer_groupby(self,group,splitter='\n'):
        for n,g in self.ddf.groupby(group):
            print(splitter)
            g.apply(rowprinter,axis=1)
        
s=storyBlock()
s.add({'ccg':ccg,'provider':provider,'txt':'Report for CCG {}, provider {}'.format(ccg,provider)})
s.add(generic_reporter(df,{'metadata':metadataDict,'header':'',
                          'cols':['Appointments'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider))


s.add(generic_reporter(df,{'metadata':metadataDict,'sort':True,'cols':apptTypes,'numFirst':True,
                          'header':'Appointment types:',
                          'prefix':"\n\t- ",'suffix':' appointments;',
                         'extractor':"Number of (.*) Appointments in the month where attended or did not attend code is  'attended on time' or 'arrived late but was seen'"},ccg,provider))


s.add(generic_reporter(df,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},ccg,provider))

s.printer()

Report for CCG 10L, provider R1F

Appointments in the month: 1195.0.

Appointment types:
	- 695.0 Cognitive Behaviour Therapy (CBT) treatment appointments;
	- 135.0 Other Low Intensity treatment appointments;
	- 90.0 Psychoeducational Peer Support treatment appointments;
	- 60.0 Guided Self Help (Computer) treatment appointments;
	- 55.0 Eye Movement Desensitisation Reprocessing treatment appointments;
	- 45.0 Behavioural Activation (Low Intensity) treatment appointments;
	- 30.0 Interpersonal Psycho Therapy (IPT) treatment appointments;
	- 20.0 Couples Therapy for Depression treatment appointments;
	- 10.0 Non - Guided Self Help (Book) treatment appointments;
	- 10.0 Guided Self Help (Book) treatment appointments;

Appointment attendance:
	- 1090.0 attended on time.
	- 30.0 cancelled appointments - patient.
	- 45.0 did not attend.
	- 30.0 arrived late but was seen.



## Comparisons

Explore comparing periods.

In [43]:
def report_rows_comparator(df,ccg,provider='all',periods=None):
    if periods is None: return report_rows(df,ccg,provider)
    if provider!='all':
        rows=df[(df['CCG']==ccg) & (df['Provider']==provider) & (df[periods[0]].isin(periods[1]))]
    else:
        rows=df[(df['CCG']==ccg) & (df[periods[0]].isin(periods[1]))]
    return rows

In [44]:
df2=pd.read_sql_query("SELECT * FROM monthly",con_iapt)
report_rows(df2,'10L','R1F',('Month',['Jun-16','May-16']))

,Month,CCG,Provider,CCGName,ProviderName,GroupType,ReferralsReceived,SelfReferrals,FirstAssessment,FirstAssessment28days,...,MeanIPTSessions,MeanCouplesSessions,MeanBehavActSessions,MeanOtherHISessions,MeanLiSessions,ImprovementRate,RecoveryRate,ReliableRecoveryRate,FirstTreatment6WeeksFinishedCourseRate,FirstTreatment18WeeksFinishedCourseRate
1085,May-16,10L,R1F,NHS ISLE OF WIGHT CCG,ISLE OF WIGHT NHS TRUST,CCG/Provider,275.0,265.0,210.0,160.0,...,NaN,NaN,NaN,9.6,5.3,64.4,52.1,50.0,95.9,100.0
2319,Jun-16,10L,R1F,NHS ISLE OF WIGHT CCG,ISLE OF WIGHT NHS TRUST,CCG/Provider,325.0,310.0,245.0,210.0,...,NaN,10.9,NaN,13.5,5.0,63.7,49.2,46.3,92.9,100.0


In [47]:
s=storyBlock()

s.add({'ccg':ccg,'provider':provider,'txt':'Report for CCG {}, provider {}'.format(ccg,provider)})
s.add(generic_reporter(df2,{'metadata':metadataDict,'header':'',
                          'cols':['Appointments'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},ccg,provider,reference='Month'))

setup={'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'}
s.add(generic_reporter(df2,setup,ccg,provider,reference="Month"))

In [48]:
s.printer()

Report for CCG 10L, provider R1F
[10L: R1F, May-16] 
Appointments in the month: 1225.0.

[10L: R1F, Jun-16] 
Appointments in the month: 1195.0.

[10L: R1F, May-16] Appointment attendance:
	- 1100.0 attended on time.
	- 25.0 cancelled appointments - patient.
	- 5.0 cancelled or postponed by health care provider.
	- 65.0 did not attend.
	- 30.0 arrived late but was seen.

[10L: R1F, Jun-16] Appointment attendance:
	- 1090.0 attended on time.
	- 30.0 cancelled appointments - patient.
	- 45.0 did not attend.
	- 30.0 arrived late but was seen.



In [49]:
s.printer_groupby(['CCG','Provider','Month'],splitter='-----')

-----
[10L: R1F, Jun-16] 
Appointments in the month: 1195.0.

[10L: R1F, Jun-16] Appointment attendance:
	- 1090.0 attended on time.
	- 30.0 cancelled appointments - patient.
	- 45.0 did not attend.
	- 30.0 arrived late but was seen.

-----
[10L: R1F, May-16] 
Appointments in the month: 1225.0.

[10L: R1F, May-16] Appointment attendance:
	- 1100.0 attended on time.
	- 25.0 cancelled appointments - patient.
	- 5.0 cancelled or postponed by health care provider.
	- 65.0 did not attend.
	- 30.0 arrived late but was seen.



## Comparing Two Periods

Need to be able to construct sentences of the form:

- *The blah blah was NN (updownX compared to MM previously).*
- *NN (MM previously) blah blah*
- *NN (MM previously, LL a year ago) blah blah*

In [227]:
#Simple dataset
diff2=pd.DataFrame([{'id':'aa','val':1,'val2':3,'period':'jan'},{'id':'aa','val':2,'val2':4,'period':'feb'},
                   {'id':'bb','val':5,'val2':10,'period':'jan'},{'id':'bb','val':4,'val2':8,'period':'feb'},
                   {'id':'cc','val':3,'val2':6,'period':'jan'},{'id':'cc','val':3,'val2':6,'period':'feb'}])
diff2

,id,period,val,val2
0,aa,jan,1,3
1,aa,feb,2,4
2,bb,jan,5,10
3,bb,feb,4,8
4,cc,jan,3,6
5,cc,feb,3,6


In [228]:
#Assume we have just one column in diffferent periods
diff2p=diff2.pivot(index='id',columns='period',values='val')
diff2p

period,feb,jan
id,,
aa,2,1
bb,4,5
cc,3,3


In [474]:
#Report on the difference between any two values in their own cols
def comparator_onecol(row,curr,prev, diff=True, prevterm='the previous value of',prevval=False,endchar='',numformat=None):
    curr=row[curr]
    prev=row[prev]
    diffval=curr-prev
    if diffval==0:
        txt='{curr}, the same as {prevterm}{prev}{endchar}'.format(prevterm=prevterm,curr=curr,
                                                           prev= '' if not prevval else ' {}'.format(prev),
                                                           diff='' if not diff else ', {}'.format(prev),
                                                                  endchar=endchar)
    elif diffval<0:
        txt='{curr}, {diff}less than {prevterm}{prev}{endchar}'.format(prevterm=prevterm,curr=curr,
                                                        prev='' if not prevval else ' {}'.format(prev),
                                                        diff='' if not diff else '{} '.format(abs(diffval)),
                                                                      endchar=endchar)
    elif diffval>0:
        txt='{curr}, {diff}greater than {prevterm}{prev}{endchar}'.format(prevterm=prevterm,curr=curr,
                                                           prev='' if not prevval else ' {}'.format(prev),
                                                           diff='' if not diff else '{} '.format(diffval),
                                                           endchar=endchar)
    else:
        if pd.notnull(curr) and pd.isnull(prev):
            txt='{}, (no previous value given)'.format(curr)
        elif pd.isnull(curr) and pd.notnull(prev):
            txt='{} previously (no current value available)'.format(prev)
        else: txt='no values available'
    return txt

In [475]:
#Preview report
diff2p.apply(lambda x: comparator_onecol(x,'feb','jan',diff=True,prevval=True),axis=1)

id
aa    2, 1 greater than the previous value of 1
bb       4, 1 less than the previous value of 5
cc       3, the same as the previous value of 3
dtype: object

In [476]:
#How do we cope with multiple cols in current and previous states?
diff3p=diff2.pivot(index='id',columns='period')
diff3p

val     val2    
period feb jan  feb jan
id                     
aa       2   1    4   3
bb       4   5    8  10
cc       3   3    6   6

In [477]:
diff3p[['val2','val']]

val2     val    
period  feb jan feb jan
id                     
aa        4   3   2   1
bb        8  10   4   5
cc        6   6   3   3

In [478]:
diff3p[['val2','val']]['val2']

period,feb,jan
id,,
aa,4,3
bb,8,10
cc,6,6


In [479]:
diff3p[['val2','val']]['val2'].ix['bb']

period
feb     8
jan    10
Name: bb, dtype: int64

In [480]:
rows=report_rows(df2,ccg,provider,('Month',['Jun-16','May-16']) )
rows

,Month,CCG,Provider,CCGName,ProviderName,GroupType,ReferralsReceived,SelfReferrals,FirstAssessment,FirstAssessment28days,...,MeanIPTSessions,MeanCouplesSessions,MeanBehavActSessions,MeanOtherHISessions,MeanLiSessions,ImprovementRate,RecoveryRate,ReliableRecoveryRate,FirstTreatment6WeeksFinishedCourseRate,FirstTreatment18WeeksFinishedCourseRate
1085,May-16,10L,R1F,NHS ISLE OF WIGHT CCG,ISLE OF WIGHT NHS TRUST,CCG/Provider,275.0,265.0,210.0,160.0,...,NaN,NaN,NaN,9.6,5.3,64.4,52.1,50.0,95.9,100.0
2319,Jun-16,10L,R1F,NHS ISLE OF WIGHT CCG,ISLE OF WIGHT NHS TRUST,CCG/Provider,325.0,310.0,245.0,210.0,...,NaN,10.9,NaN,13.5,5.0,63.7,49.2,46.3,92.9,100.0


In [481]:
cols=['FirstAssessment','FirstAssessment28days']
core=['CCG','Provider']
rows[core+cols+['Month']].set_index(core).pivot(columns='Month')

FirstAssessment        FirstAssessment28days       
Month                 Jun-16 May-16                Jun-16 May-16
CCG Provider                                                    
10L R1F                245.0  210.0                 210.0  160.0

In [482]:
rows[core+cols+['Month']].set_index(core).pivot(columns='Month')['FirstAssessment28days'].reset_index()

Month,CCG,Provider,Jun-16,May-16
0,10L,R1F,210.0,160.0


In [483]:
'''def series_lister_item_comparator(numFirst,cnt,typ,prefix,suffix,extractor,replacer,lower):
    if extractor is not None:
        rs=re.search(extractor,typ)
        if rs:
            typ=rs.group(1)
    if replacer is not None:
        typ=re.sub(replacer[0],replacer[1],typ)
    if numFirst:
        txt='{prefix}{cnt} {typ}{suffix}'.format(cnt=cnt,typ=ls(typ) if lower else typ,
                                                      prefix=prefix,suffix=suffix)
    else:
        txt='{prefix}{typ}: {cnt}{suffix}'.format(cnt=cnt,typ=typ,
                                                       prefix=prefix,suffix=suffix)
    return txt'''

from collections import OrderedDict

def series_lister_nonzero_comparator(row,period,cols,metadata,sort=False,numFirst=False,prefix='\n',suffix='',
                          header='',extractor=None,replacer=None,lower=False):
    txt=header
    
    #get values for each col, then order cols
    od={}
    for i in cols:
        if pd.notnull(row[i][period[1][0]]) and pd.notnull(row[i][period[1][1]]):
            od[i]=row[i][period[1][0]]
    cols=od
    if sort:
        cols=OrderedDict(sorted(od.items()))
    for i in cols:
        typ=metadata[i]
        val=comparator_onecol(row[i],period[1][0],period[1][1],diff=True,prevval=True)
        txt='{txt}{item}'.format(txt=txt,item=series_lister_item(numFirst,val,typ,prefix,suffix,extractor,replacer,lower))      
    return txt+'\n'

def generic_reporter_comparator(df,setup,ccg,provider='all',reference=None,period=None):
    rows=report_rows(df,ccg,provider,period)

    core=['CCG','Provider']
    cols=setup['cols']
    rows=rows[core+cols+[period[0]]].set_index(core).pivot(columns=period[0])

    #if isinstance(reference, str): core.append(reference)

    tmp=rows.copy()

    tmp['txt']= rows.apply(lambda x: series_lister_nonzero_comparator(x,period,**setup),axis=1)
    tmp=tmp.reset_index()
    

    #Need to identify multiple reports
    if reference is None and len(rows)>1: reference=True
    if reference: tmp['txt']=tmp.apply(lambda x: '[{}: {}{}] {}'.format(x['CCG'],x['Provider'],
                                                                        '', #if not isinstance(reference, str) else ', {}'.format(x[reference]),
                                                                        x['txt']),axis=1)
    tmp=tmp[core+['txt']]
    tmp.columns = tmp.columns.get_level_values(0)
    return tmp

In [484]:
setup['sort']=False
xx=generic_reporter_comparator(df2,setup,ccg,provider,period=('Month',['Jun-16','May-16']))
xx

,CCG,Provider,txt
0,10L,R1F,"Appointment attendance:\n\t- 1090.0, 10.0 less than the previous value of 1100.0 attended on time.\n\t- 30.0, 5.0 greater than the previous value of 25.0 cancelled appointments - patient.\n\t- 45.0, 20.0 less than the previous value of 65.0 did not attend.\n\t- 30.0, the same as the previous value of 30.0 arrived late but was seen.\n"


In [485]:
setup['sort']=True
generic_reporter_comparator(df2,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},ccg,provider,
                period=('Month',['Jun-16','May-16']))

,CCG,Provider,txt
0,10L,R1F,"Appointment attendance:\n\t- did not attend: 45.0, 20.0 less than the previous value of 65.0.\n\t- attended on time: 1090.0, 10.0 less than the previous value of 1100.0.\n\t- cancelled appointments - patient: 30.0, 5.0 greater than the previous value of 25.0.\n\t- arrived late but was seen: 30.0, the same as the previous value of 30.0.\n"


In [486]:
type(generic_reporter_comparator(df2,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':True,'prefix':"\n\t- ",'suffix':'.'},
                       ccg,provider,period=('Month',['Jun-16','May-16'])))

pandas.core.frame.DataFrame

In [487]:
s=storyBlock()
s.add({'ccg':ccg,'provider':provider,'txt':'Report for CCG {}, provider {}'.format(ccg,provider)})
s.add(generic_reporter_comparator(df2,{'metadata':metadataDict,'header':'',
                          'cols':['Appointments'],'sort':True,'numFirst':False,'prefix':"\n",'suffix':'.'},
                       ccg,provider,period=('Month',['Jun-16','May-16'])))


s.add(generic_reporter_comparator(df2,{'metadata':metadataDict,'sort':True,'cols':apptTypes,'numFirst':False,
                          'header':'Appointment types:',
                          'prefix':"\n\t- ",'suffix':' appointments;',
                         'extractor':"Number of (.*) Appointments in the month where attended or did not attend code is  'attended on time' or 'arrived late but was seen'"},
                       ccg,provider,period=('Month',['Jun-16','May-16'])))


s.add(generic_reporter_comparator(df2,{'metadata':metadataDict,'header':'Appointment attendance:',
                          'extractor':"Appointments in the month where attended or did not attend code is '(.*)'+?",
                          'cols':appts,'numFirst':False,'prefix':"\n\t- ",'suffix':'.'},
                       ccg,provider,period=('Month',['Jun-16','May-16'])))

s.printer()

Report for CCG 10L, provider R1F

Appointments in the month: 1195.0, 30.0 less than the previous value of 1225.0.

Appointment types:
	- Behavioural Activation (Low Intensity) treatment: 45.0, 15.0 greater than the previous value of 30.0 appointments;
	- Cognitive Behaviour Therapy (CBT) treatment: 695.0, 50.0 less than the previous value of 745.0 appointments;
	- Couples Therapy for Depression treatment: 20.0, 15.0 less than the previous value of 35.0 appointments;
	- Eye Movement Desensitisation Reprocessing treatment: 55.0, 10.0 less than the previous value of 65.0 appointments;
	- Guided Self Help (Computer) treatment: 60.0, 5.0 greater than the previous value of 55.0 appointments;
	- Interpersonal Psycho Therapy (IPT) treatment: 30.0, 15.0 greater than the previous value of 15.0 appointments;
	- Non - Guided Self Help (Book) treatment: 10.0, the same as the previous value of 10.0 appointments;
	- Other Low Intensity treatment: 135.0, 20.0 less than the previous value of 155.0 appo